In [1]:
import numpy as np    
import pandas as pd    
import matplotlib.pyplot as plt  
import os  
import cartopy.crs as ccrs  
import cartopy.feature as cfeature  
import matplotlib.ticker as mticker  
from matplotlib.ticker import MultipleLocator, FuncFormatter
import xarray as xr  

In [2]:
df_0 = pd.read_excel(r'D:\ze_1112\最新数据\NORC2022-09_Chl-a+营养盐_20240422更新_绘图使用数据.xls')
# df.dropna(inplace=True)
df_0 = df_0.rename(columns={'站位':'站位名'})
stations = []
for i in df_0['站位名']:
    stations.append(i)
stations = list(set(stations))
print(stations)

['E130-14', 'E130-07', 'E140-07', 'N03-02', 'N21-14', 'E163-06s', 'QB08', 'E140-11', 'E163-05', 'E130-02', 'E130-08', 'E130-05', 'N21-10', 'E140-03orQB01', 'E130-06', 'QB06', 'N21-11', 'E130-04', 'E163-04s', 'E163-04', 'E130-03', 'N03-04', 'N03-03', 'QB04', 'N21-12', 'E163-01', 'E163-02', 'E163-07', 'QB07', 'FB01', 'E130-12', 'N21-13', 'E130-09', 'QB03', 'E130-13', 'E140-08', 'E163-03', 'E163-06', 'E130-11', 'E130-10']


In [59]:
df_q = xr.open_dataset(r'G:\data\4093\cmems_mod_glo_bgc_my_0.25deg_P1M-m_1721905957896.nc')
df_q = df_q.sel(depth=4000, method = 'nearest')
df_q = df_q.sel(time=df_q['time.month'] == 4) # 季节选取
df_q = df_q.mean(dim='time') # 季节平均
df_q

<xarray.Dataset> Size: 157kB
Dimensions:    (latitude: 121, longitude: 161)
Coordinates:
    depth      float32 4B 4.093e+03
  * latitude   (latitude) float32 484B -5.0 -4.75 -4.5 -4.25 ... 24.5 24.75 25.0
  * longitude  (longitude) float32 644B 125.0 125.2 125.5 ... 164.5 164.8 165.0
Data variables:
    o2         (latitude, longitude) float32 78kB nan nan nan ... 160.5 160.4
    si         (latitude, longitude) float32 78kB nan nan nan ... 148.2 148.3

In [4]:
df_2024 = pd.read_excel(r'D:\ze_1112\最新数据\NORC2022-09_Chl-a+营养盐_20240422更新_绘图使用数据.xls')
# df.dropna(inplace=True)
df_2024 = df_2024.rename(columns={'站位':'站位名'})
df_2021 = pd.read_excel(r'E:\all_draw\CTD\营养盐数据.xlsx')
df_2021['站位'] = df_2021['站位'].fillna(method='ffill')
df_2021 = df_2021.rename(columns={'站位':'站位名'})


C:\Users\Administrator\AppData\Local\Temp\ipykernel_19616\2144448703.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_2021['站位'] = df_2021['站位'].fillna(method='ffill')


In [5]:
stations_2021 = []
for i in df_2021['站位名']:
    stations_2021.append(i)
stations_2021 = list(set(stations_2021))
print(stations_2021)
stations_2024 = []
for i in df_2024['站位名']:
    stations_2024.append(i)
stations_2024 = list(set(stations_2024))
print(stations_2024)

['E163-08', 'QB16', 'QB08', 'N3-05', 'E163-05', 'E142-5', 'QB12', 'QB06', 'QB15', 'E163-04', 'QB02', 'QB18', 'QB-07new', 'QB11', 'QB04', 'QB09', 'E163-01', 'QB01', 'FB01', 'E163-07', 'QB07', 'E163-02', nan, 'QB05', 'EQ-03', 'E142-07', 'QB17', 'QB03', 'E163-09', 'QB14', 'E142-06', 'QB10', 'E163-11', 'E163-10', 'Mk-03', 'FB02', 'E163-03', 'E163-06', 'N3-03', 'QB19', 'E142-10', 'N3-01']
['E130-14', 'E130-07', 'E140-07', 'N03-02', 'N21-14', 'E163-06s', 'QB08', 'E140-11', 'E163-05', 'E130-02', 'E130-08', 'E130-05', 'N21-10', 'E140-03orQB01', 'E130-06', 'QB06', 'N21-11', 'E130-04', 'E163-04s', 'E163-04', 'E130-03', 'N03-04', 'N03-03', 'QB04', 'N21-12', 'E163-01', 'E163-02', 'E163-07', 'QB07', 'FB01', 'E130-12', 'N21-13', 'E130-09', 'QB03', 'E130-13', 'E140-08', 'E163-03', 'E163-06', 'E130-11', 'E130-10']


In [6]:
import re

def dms_to_decimal(dms):
    # 使用正则表达式匹配度分秒格式，包括不同的分隔符和方向指示符
    match = re.match(r"(\d+)[°\s](\d+)[′'’\s](\d+(\.\d+)?)[\"\s]?(N|S|E|W)?", dms)
    if match:
        degrees = float(match.group(1))
        minutes = float(match.group(2))
        seconds = float(match.group(3))
        direction = match.group(5)
        decimal = degrees + (minutes / 60) + (seconds / 3600)
        if direction in ['S', 'W']:
            decimal = -decimal
        return decimal
    else:
        raise ValueError(f"无法解析的度分秒格式: {dms}")

# 自定义刻度格式化函数
def lon_formatter(x, pos):
    if x < 0:
        return f'{abs(int(x))}°W'
    elif x > 180:
        return f'{360-int(x)}°W'
    else:
        return f'{int(x)}°E'

def lat_formatter(x, pos):
    if x > 0:
        return f'{int(x)}°N' 
    else:
        return f'{int(x)}°S'

def dms_to_decimal_2024(dms):
    """将度分秒转换为十进制度数"""
    dms = dms.replace('°', ' ').replace('′', ' ').replace('″', ' ').replace('"E', ' ').replace('"N', ' ').replace('"S', ' ')
    parts = dms.split()
    degrees = float(parts[0])
    minutes = float(parts[1]) if len(parts) > 1 else 0
    seconds = float(parts[2]) if len(parts) > 2 else 0
    decimal = degrees + (minutes / 60) + (seconds / 3600)
    return decimal

In [7]:
def dms_to_decimal(dms):
    """将度分秒转换为十进制度数"""
    dms = dms.replace('°', ' ').replace('′', ' ').replace('″', ' ').replace('"E', ' ').replace('"N', ' ').replace('"S', ' ')
    parts = dms.split()
    degrees = float(parts[0])
    minutes = float(parts[1]) if len(parts) > 1 else 0
    seconds = float(parts[2]) if len(parts) > 2 else 0
    decimal = degrees + (minutes / 60) + (seconds / 3600)
    return decimal

In [60]:
stations_ctd = os.listdir(r'D:\ze_1112\最新数据\CTD补上溶解氧_04共享航次计划调查资料汇交表')
longitude_q = df_q['longitude'].values
latitude_q = df_q['latitude'].values
sio4_q = df_q['si'].values
o2_q = df_q['o2'].values
o2_q = np.where(o2_q < 110, np.nan, o2_q)

plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
plt.rcParams['font.size'] = 22

    
fig = plt.figure(figsize=(10, 8))
ax1 = fig.add_subplot(111, projection=ccrs.PlateCarree(), facecolor='white')    
ax1.set_extent([125, 165.001, -5, 25.001], crs=ccrs.PlateCarree())
qht = ax1.contourf(longitude_q, latitude_q, o2_q, levels=60, cmap='RdYlBu_r', transform=ccrs.PlateCarree(), vmin=150, vmax=180)
cbar2 = plt.colorbar(qht, ax=ax1, orientation='vertical', pad=0.02, aspect=30, shrink=0.65)
cbar2.set_label('o2-contourf  (μmol/L)', fontsize=15)
cbar2.set_ticks(np.linspace(150, 181, 4))  # 设置刻度范围与 qht 一致
cbar2.set_ticklabels([f'{tick:.0f}' for tick in np.linspace(150, 180, 4)], fontsize=22)  # 设置刻度标签

for i in stations:
    df = pd.read_excel(os.path.join(rf'D:\ze_1112\最新数据\CTD补上溶解氧_04共享航次计划调查资料汇交表\CTD-{i}.xls'), skiprows=3, usecols=[1,5,6,8,11,13,16], names=['站位名', '纬度', '经度', '深度', '水温', '盐度', '溶解氧'])
    df = df.drop(0)
    max_depth = df['深度'].max()
    df = df[df['深度'] == 4000]
    longitude = df['经度'].apply(dms_to_decimal).values # type: ignore
    latitude = df['纬度'].apply(dms_to_decimal).values  # type: ignore
    name = df['站位名'].values
    o2 = df['溶解氧'].values
    print(o2)
    depth = df['深度'].values
    
    sc = ax1.scatter(longitude, latitude, s=80, c=o2, cmap='RdYlBu_r', marker='^', alpha=1, edgecolors='k' , transform=ccrs.PlateCarree(), zorder=10,vmin=110, vmax=120)

ax1.spines[:].set_linewidth(2)
ax1.add_feature(cfeature.BORDERS, linewidth=2)
ax1.add_feature(cfeature.COASTLINE, linewidth=2)
ax1.add_feature(cfeature.RIVERS, linewidth=2)
ax1.add_feature(cfeature.LAND, edgecolor='black', zorder=9)

import xarray as xr
ds = xr.open_dataset(r"E:\all_draw\data\Copernicus_bathy\cmems_mod_glo_phy_my_0.083deg_static_1720691864705.nc")
elevation = ds['deptho'].values
longitude = ds['longitude'].values
latitude = ds['latitude'].values
mask = elevation > 4000
mask_1 = elevation >2000
mask_2 = elevation > 4000
# zhe_gai = ax1.contourf(longitude, latitude, mask, levels=[0, 0.5], colors='#C0C0C0', transform=ccrs.PlateCarree(), alpha = 1)
zhe_gai_2 = ax1.contourf(longitude, latitude, mask_2, levels=[0, 0.5], colors='#808A87', transform=ccrs.PlateCarree(), alpha = 1)
# zhe_gai_1 = ax1.contourf(longitude, latitude, mask_1, levels=[0, 0.5], colors='#E3A869', transform=ccrs.PlateCarree(), alpha = 0.8)

ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
gl = ax1.gridlines(draw_labels=False, linewidth=1, color='white', alpha=0.15, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 13, 'color': 'black'}
gl.ylabel_style = {'size': 13, 'color': 'black'}
ax1.set_xticks(np.arange(125, 165.001, 10))
ax1.set_yticks(np.arange(-5, 25.001, 5))
ax1.xaxis.set_major_formatter(FuncFormatter(lon_formatter))
ax1.yaxis.set_major_formatter(FuncFormatter(lat_formatter))
    
cbar = plt.colorbar(sc, ax=ax1, orientation='vertical', pad=0.02, aspect=30, shrink=0.65)
cbar.set_label('o2-scatters  (μmol/L)', fontsize=15)
# cbar.set_ticks(np.arange(60, 76, 5))
# cbar.set_ticklabels(['60', '65', '70', '75'], fontsize=22)
from matplotlib.lines import Line2D
legend_elements = [Line2D([0], [0], marker='^', color='w', label='2023 Voyage',
                          markerfacecolor='none', markeredgecolor='k', markersize=10),
                   Line2D([0], [0], marker='s', color='#808A87', label='4000m Terrain',markersize=14, linestyle='none')]
ax1.legend(handles=legend_elements, loc='upper right', fontsize=12,framealpha=0.8, fancybox=True)

ax1.set_title('2023 Monthly Average(April) O2 at 4000m Depth', fontsize=20, y=1.05)
# plt.show()    
plt.savefig('2023Monthly_o2_4000.png', bbox_inches='tight', dpi=300)
plt.close()

[114.84006076]
[115.20746528]
[116.41970486]
[]
[112.16649306]
[]
[109.28402778]
[116.39648437]
[126.83415799]
[114.43463542]
[114.89131944]
[114.54040799]
[112.41137153]
[117.25421007]
[114.7358941]
[117.00412326]
[112.6047309]
[114.83203125]
[]
[128.52690972]
[114.9687066]
[116.46436632]
[]
[]
[113.21992187]
[]
[]
[123.22152778]
[116.08511285]
[116.57057292]
[114.84105903]
[112.64631076]
[114.67434896]
[]
[115.10221354]
[]
[]
[126.78728299]
[114.93446181]
[115.16957465]
